# Mexican federal budget pre-processing pipeline

## Instructions

To you run the notebook:

1. choose a unique `ITERATION_LABEL` for each pipeline run
2. specify and describe your input files (`INPUT_FILES`)
3. make sure your column mapping (`COLUMN_ALIASES`) is correct
3. run the whole notebook by clicking on __Kernel > Restart & Run All__

## Settings

Choose a unique iteration label for each pipeline run.

In [44]:
ITERATION_LABEL = 'iteration-7-development'

Put your input files inside the `pipeline.in` folder and describe them here.

In [2]:
INPUT_FILES = {
    2010: {'name': 'Cuenta_Publica_2010.csv', 'encoding': 'windows-1252'},
    2011: {'name': 'Cuenta_Publica_2011.csv', 'encoding': 'windows-1252'},
    2012: {'name': 'Cuenta_Publica_2012.csv', 'encoding': 'windows-1252'},
    2013: {'name': 'Cuenta_Publica_2013.csv', 'encoding': 'windows-1252'},
    2014: {'name': 'Cuenta_Publica_2014.csv', 'encoding': 'windows-1252'},
    2015: {'name': 'Cuenta_Publica_2015.csv', 'encoding': 'windows-1252'},
    2016: {'name': 'PEF2016_AC01.csv', 'encoding': 'cp850'}
}

If your input files don't all have the same column names, define your mapping here. 

In [3]:
COLUMN_ALIASES = {
    'Actividad Institucional': ['AI'],
    'Adefas': ['ADEFAS'],
    'Aprobado': [
        'PEF_2016',
        'Importe Presupuesto de Egresos de la Federación',
        'Importe Presupuesto de Egresos de la Federación (PEF)'
    ],
    'Ciclo': None,
    'Clave de cartera': ['CLAVE_CARTERA'],
    'Descripción de Fuente de Financiamiento': ['FUENTE_FINAN_DESCRIPCION'],
    'Descripción de Función': ['FUNCIONL_DESCRIPCION'],
    'Descripción de Grupo Funcional': [
        'Descripción de Finalidad',
        'GRUPO_FUN_DESCRIPCION',
        'Descripción de Grupo Funcional'
    ],
    'Descripción de Objeto del Gasto': ['CONCEPTO_DESCRIPCION'],
    'Descripción de Programa Presupuestario': ['PROGR_PRES_DESCRIPCION'],
    'Descripción de Ramo': ['RAMO_DESCRIPCION'],
    'Descripción de Reasignacion': ['REASIGNACION_DESCRIPCION'],
    'Descripción de Subfunción': ['SUBFUNCIONL_DESCRIPCION'],
    'Descripción de Tipo de Gasto': ['TIPO_GASTO_DESCRIPCION'],
    'Descripción de Unidad Responsable': ['UNIDAD_DESCRIPCION'],
    'Descripción de la Actividad Institucional': [
        'ACTIVIDAD_INST_DESCRIPCION',
        'Descripción de Actividad Institucional'
    ],
    'Descripción de la entidad federativa': ['ENTIDAD_FED_DESCRIPCION'],
    'Descripción de la modalidad del programa presupuestario': [
        'MODALIDAD_DESCRIPCION',
        'Descripción del Identificador del Programa Presupuestario',
        'Descripción del Identificador de Programa Presupuestario'
    ],
    'Devengado': None,
    'Ejercicio': None,
    'Ejercido': None,
    'Entidad Federativa': ['EF'],
    'Fuente de Financiamiento': ['FF'],
    'Función': ['FN'],
    'Grupo Funcional': [
        'Finalidad', 'GF', 'Grupo Funcional'
    ],
    'Modalidad del Programa presupuestario': [
        'MOD',
        'Identificador de Programa Presupuestario',
        'Identificador del Programa Presupuestario'
    ],
    'Modificado': None,
    'Objeto del Gasto': ['CONCEPTO'],
    'Pagado': None,
    'Programa Presupuestario': ['PP'],
    'Ramo': None,
    'Reasignacion': ['RA'],
    'Subfunción': ['SF'],
    'Tipo de Gasto': ['TG'],
    'Unidad Responsable': ['UNIDAD']
}

The following hierarchical categories will have IDs prefixed with the parent categories:

In [4]:
HIERARCHIES = {
    'functional': [
        'Grupo Funcional', 
        'Función', 
        'Subfunción', 
        'Actividad Institucional'
    ],
#     'administrative': [
#         'Ramo', 
#         'Unidad Responsable'
#     ],
#     'activities': [
#         'Modalidad del Programa presupuestario', 
#         'Programa Presupuestario'
#     ],
}

That's it. Now just run the notebook from beginning to end.

## Imports

In [40]:
from sys import stdout
from pandas import read_csv, concat, DataFrame, ExcelWriter, ExcelFile, Series
from numpy import nan
from os.path import join, isdir
from os import mkdir
from json import dumps, loads
from pprint import pprint
from slugify import slugify

## Configuration

In [6]:
BASENAME = 'mexican_federal_budget'
INPUT_FOLDER = 'pipeline.in'
OUTPUT_FOLDER = 'pipeline.out'
ITERATION_FOLDER = join(OUTPUT_FOLDER, ITERATION_LABEL)
MERGED_FILE = join(ITERATION_FOLDER, BASENAME + '.merged.csv')
CATALOGS_FOLDER = 'objeto_del_gasto.catalog'
CATALOGS_FILE = 'objeto_del_gasto.catalog.xlsx'

In [7]:
if isdir(ITERATION_FOLDER):
    raise ValueError('Please enter a unique iteration label')
    
mkdir(ITERATION_FOLDER)

## Encoding inspection

Detect the file encodings of the input files using the `cChardet` utility library. __Warning:__ it's not always accurate. This is meant only as an indication only. In the end, encodings will be taken from `INPUT_FILES`.

In [8]:
def detect_encodings():
    """Detect CSV file encoding with the cChardet library"""

    try:
        import cchardet as chardet
    except ImportError:
        cChardet = 'https://github.com/PyYoshi/cChardet'
        print('Encoding inspection skipped: install %s', cChardet)
        return

    results = {}
    results_file = join(OUTPUT_FOLDER, ITERATION_LABEL, 'encodings.detected.json')
    
    for year, file in sorted(INPUT_FILES.items()):
        datafile = join(INPUT_FOLDER, file['name'])
        
        with open(datafile, 'rb') as f:
            text = f.read()
            
        result = chardet.detect(text)
        results.update({year: result})
        print(year, 'Inspected', file['name'], result)
    
    with open(results_file, 'w+') as json:
        json.write(dumps(results, indent=4))
        print('\nSaved encoding detection report to', results_file)
        
# detect_encodings()

## Load files

In [9]:
def read_columns(file, encoding):
    """Return clean CSV file headers"""
    
    with open(file, encoding=encoding) as csv:
        header = csv.readline()
        return header.replace('\n', '').split(',')

In [10]:
def force_strings(columns):
    """Return string enforcement for each column of a CSV file"""
    
    for column in columns:
        yield column, str

In [11]:
def load_csv_files():
    """Load raw data (CSV) files"""
    
    batch = {}
    
    for year, file in sorted(INPUT_FILES.items()):
        filepath = join(INPUT_FOLDER, file['name'])
        column_names = read_columns(filepath, file['encoding'])
        column_types = dict(force_strings(column_names))
        
        batch[year] = read_csv(filepath, encoding=file['encoding'], dtype=column_types)
        print('Loaded', file['name'], 'with encoding', file['encoding'])
        stdout.flush()
            
    return batch

## Clean the data

In [12]:
def strip_cell_padding(batch):
    for year in sorted(batch.keys()):
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
            batch[year][column] = batch[year][column].apply(lambda x: x.strip() if x is not nan else x)
        print(year, 'stripped cell paddings')
        stdout.flush()

In [13]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    del batch[year][column]
                    print(year, column, 'deleted')
                    stdout.flush()
                except KeyError:
                    pass  

In [14]:
def count_missing_values(batch):
    collector = {}
    table = []

    for column in get_union_of_columns(batch):
        row = {'Column': column}
        collector.update({column: []})
        
        for year in batch.keys():
            if column in batch[year].columns:
                is_empty = batch[year][column].isnull()
                empty_lines = batch[year].where(is_empty).dropna(how='all')
                collector[column].extend(empty_lines.to_dict(orient='records'))
                nb_empty_cells = len(empty_lines)
            else:
                nb_empty_cells = nan
                
            row.update({year: nb_empty_cells})
            if nb_empty_cells not in (nan, 0):
                print(year, 'found', nb_empty_cells, 'missing values in', column)

        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    empty_values_overview_table = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    
    return empty_values_overview_table, collector

In [15]:
def count_duplicates(batch):
    for year, df in sorted(batch.items()):
        nb_duplicate_lines = df.duplicated().apply(lambda x: 1 if x is True else 0).sum()
        print(year, 'found', nb_duplicate_lines, 'duplicate lines')

## Alias column names

In [16]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [17]:
from yaml import load

def load_aliases(file):
    with open(file) as yaml:
        aliases = load(yaml.read())
        return aliases

In [18]:
def map_columns_to_aliases(batch, list_of_aliases):
    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, column, 'replaced with', reference)
                            stdout.flush()
                            break  
                else:
                    print(year, 'NO ALIAS: ', column)
                    stdout.flush()

In [19]:
def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    overview = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    return overview

## Check expenditure sums

There's a little cleaning to do on the amount columns (zeros represented by a dash). Assume thousands are seperated by a comma.

In [20]:
EXPENDITURE_COLUMNS = [
    'Ejercido', 
    'Devengado', 
    'Aprobado', 
    'Pagado', 
    'Modificado', 
    'Adefas', 
    'Ejercicio'
]

def clean_expenditure_columns(batch):
    check_sums = []

    for column in EXPENDITURE_COLUMNS:
        row = {'Column': column}
        
        for year in sorted(batch.keys()):
            try:
                series = batch[year][column]
                
                # I'm assuming -' represents zero
                series = series.apply(lambda x: '0' if x == '-' else x)
                series = series.apply(lambda x: x.replace(',', '') if x is not nan else x)                
                batch[year][column] = series.astype(float)
                check_sum = batch[year][column].sum()
                
                print(year, 'cleaned and summed', column, '=', check_sum, 'pesos')
                
            except KeyError:
                check_sum = nan
                
            row.update({year: check_sum})
        
        check_sums.append(row)

    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    return DataFrame(check_sums).reindex_axis(ordered_columns, axis=1)    

## Objeto del Gasto Column split

In [21]:
from os.path import join

def generate_catalog(file):
    
    catalog_ = {}
    catalog_file = ExcelFile(file)
    INDEX_COLUMN = 0
    
    for sheet in catalog_file.sheet_names:
        if sheet != 'Concatenated':
            name = slugify(sheet, separator='_')
            output = join('objeto_del_gasto.catalog', name + '.csv')

            df = catalog_file.parse(sheet).dropna()
            index = df.columns[INDEX_COLUMN]

            df[index] =  df[index].astype(str)
            df.set_index(index, inplace=True)
            df = df.groupby(df.index).first()
            df.sort_index(inplace=True)
            
            message = 'Loaded catalog {sheet} into "{name}" ({nb} lines)'
            parameters = dict(sheet=sheet, name=name, nb=len(df))

            print(message.format(**parameters))
            catalog_[name] = df['DESCRIPCION']
    
    print()
    return catalog_

In [22]:
def split_objeto_del_gasto(batch):
    catalog = generate_catalog(CATALOGS_FILE)
    missing_in_catalog = []

    def has_digits(n, N):
        return n is not nan and len(n) >= N 

    def lookup(n, table, year):
        try:
            return catalog[table].loc[n]
        except KeyError:
            missing_in_catalog.append({'year': year, 'table': table, 'ID': n})
            return nan
        except TypeError:
            # n is nan
            return nan


    for year in sorted(batch.keys()):
        print(year, 'splitting objeto del gasto column')
        objeto = batch[year]['Objeto del Gasto'].astype(str)
        
        batch[year]['Capitulo'] = objeto.apply(lambda x: x[0] + '000' if x not in (nan, 'nan') else nan)
        batch[year]['Concepto'] = objeto.apply(lambda x: x[:2] + '00' if x not in (nan, 'nan') else nan)
        batch[year]['Partida Genérica'] = objeto.apply(lambda x: x[:3] if has_digits(x, 4) else nan)
        batch[year]['Partida Específica'] = objeto.apply(lambda x: x if has_digits(x, 5) else nan)

        batch[year]['Descripción de Capitulo'] = batch[year]['Capitulo'].map(lambda x: lookup(x, 'capitulo', year))  
        batch[year]['Descripción de Concepto'] = batch[year]['Concepto'].map(lambda x: lookup(x, 'concepto', year))  
        batch[year]['Descripción de Partida Genérica'] = batch[year]['Partida Genérica'].map(lambda x: lookup(x, 'partida_generica', year))  
        batch[year]['Descripción de Partida Específica'] = batch[year]['Partida Específica'].map(lambda x: lookup(x, 'partida_especifica', year))  
        
    return DataFrame(missing_in_catalog).drop_duplicates()

## Prefix IDs 
Disambiguating sub-categories may require prefixing their IDs with their parents' IDs.

In [23]:
def prefix_ids(batch):
    for year in batch.keys():       
        for hierarchy, levels in HIERARCHIES.items():
            prefix = batch[year]['Ciclo'].apply(lambda x: '')
            for n, level in enumerate(levels):
                dash = '-' if n > 0 else ''
                prefix = prefix + dash + batch[year][level]  
                batch[year][level] = prefix
                
                print(year, 'prefixed', hierarchy, 'level', n, level)
                stdout.flush()

##  Pipeline

In [24]:
def do_pipeline():

    def echo_section(section):
        print('\n', section, '\n')

    echo_section('Loading files')
    datasets = load_csv_files()
    
    echo_section('Delete empty columns')
    delete_empty_columns(datasets)

    echo_section('Stripping padding from cells')
    strip_cell_padding(datasets)
    
    echo_section('Counting duplicate lines (NOT de-duplicating)')
    count_duplicates(datasets)
    
    echo_section('Mapping column to aliases')
    map_columns_to_aliases(datasets, COLUMN_ALIASES)

    echo_section('Counting missing values')
    missing_values_report, bad_records = count_missing_values(datasets)
    
    echo_section('Building column mapping overview')
    column_mapping_report = build_overview(datasets)
    
    echo_section('Cleaning expenditure columns')
    sums_report = clean_expenditure_columns(datasets)
    
    echo_section('Breaking down Objeto del Gasto column')
    missing_catalog_ids = split_objeto_del_gasto(datasets)
        
    echo_section('Prefixing IDs of certain category hierarchies')
    prefix_ids(datasets)

    echo_section('Saving pipeline configuration')

    reports_file = join(ITERATION_FOLDER, BASENAME + '.reports.xlsx')
    writer = ExcelWriter(reports_file)    
    missing_values_report.to_excel(writer, 'missing values', encoding='utf-8', index=False)
    column_mapping_report.to_excel(writer, 'column mapping', encoding='utf-8', index=False)
    sums_report.to_excel(writer, 'check sums', encoding='utf-8', index=False)
    missing_catalog_ids.to_excel(writer, 'missing_catalog_IDs', encoding='utf-8', index=False)    
    print('Saved 4 reports to', reports_file)    

    aliases_file = join(ITERATION_FOLDER, BASENAME + '.aliases.json')
    inputs_file = join(ITERATION_FOLDER, BASENAME + '.inputs.json')
    levels_file = join(ITERATION_FOLDER, BASENAME + '.levels.json')
    bad_records_file = join(ITERATION_FOLDER, BASENAME + '.missing.json')

    with open(bad_records_file, 'w+') as json:
        json.write(dumps(bad_records, indent=4))
        
    with open(aliases_file, 'w+') as json:
        json.write(dumps(COLUMN_ALIASES, indent=4))
        
    with open(levels_file, 'w+') as json:
        json.write(dumps(HIERARCHIES, indent=4))
        
    with open(inputs_file, 'w+') as json:
        json.write(dumps(INPUT_FILES, indent=4))
    
    print('Saved input configuration to', inputs_file)    
    print('Saved column aliases to', aliases_file) 
    print('Saved bad records (those with empty cells) to', bad_records_file)    
    print('Saved hierarchy levels used for prefixing to', levels_file) 
    
    echo_section('Pipeline run "%s" done' % ITERATION_LABEL)

    return missing_catalog_ids, column_mapping_report, missing_values_report, sums_report, datasets

## Run the pipeline

In [25]:
missing_ids, column_mapping, missing_values, sums, budgets = do_pipeline()


 Loading files 

Loaded Cuenta_Publica_2010.csv with encoding windows-1252
Loaded Cuenta_Publica_2011.csv with encoding windows-1252
Loaded Cuenta_Publica_2012.csv with encoding windows-1252
Loaded Cuenta_Publica_2013.csv with encoding windows-1252
Loaded Cuenta_Publica_2014.csv with encoding windows-1252
Loaded Cuenta_Publica_2015.csv with encoding windows-1252
Loaded PEF2016_AC01.csv with encoding cp850

 Delete empty columns 

2011 Unnamed: 25 deleted
2011 Unnamed: 26 deleted
2011 Unnamed: 27 deleted
2011 Unnamed: 28 deleted
2011 Unnamed: 29 deleted
2011 Unnamed: 30 deleted
2011 Unnamed: 31 deleted
2011 Unnamed: 32 deleted
2011 Unnamed: 33 deleted
2011 Unnamed: 34 deleted
2011 Unnamed: 35 deleted
2011 Unnamed: 36 deleted
2011 Unnamed: 37 deleted
2011 Unnamed: 38 deleted
2011 Unnamed: 39 deleted
2011 Unnamed: 40 deleted
2011 Unnamed: 41 deleted

 Stripping padding from cells 

2010 stripped cell paddings
2011 stripped cell paddings
2012 stripped cell paddings
2013 stripped cell padd

In [26]:
from gc import collect
collect()

2486

In [27]:
for year, budget in budgets.items():
    filepath = MERGED_FILE.replace('merged', str(year))
    budget.to_csv(filepath, encoding='utf-8', index=False)
    print('Saved', filepath)
    stdout.flush()

Saved pipeline.out/iteration-7-development/mexican_federal_budget.2016.csv
Saved pipeline.out/iteration-7-development/mexican_federal_budget.2010.csv
Saved pipeline.out/iteration-7-development/mexican_federal_budget.2011.csv
Saved pipeline.out/iteration-7-development/mexican_federal_budget.2012.csv
Saved pipeline.out/iteration-7-development/mexican_federal_budget.2013.csv
Saved pipeline.out/iteration-7-development/mexican_federal_budget.2014.csv
Saved pipeline.out/iteration-7-development/mexican_federal_budget.2015.csv


In [28]:
merged = concat(list(budgets.values()))
merged.to_csv(MERGED_FILE, encoding='utf-8', index=False)
print('Saved merged dataset to', MERGED_FILE)    

Saved merged dataset to pipeline.out/iteration-7-development/mexican_federal_budget.merged.csv


## Quality control

In [29]:
sorted(list(budget.columns))

['Actividad Institucional',
 'Adefas',
 'Aprobado',
 'Capitulo',
 'Ciclo',
 'Clave de cartera',
 'Concepto',
 'Descripción de Capitulo',
 'Descripción de Concepto',
 'Descripción de Fuente de Financiamiento',
 'Descripción de Función',
 'Descripción de Grupo Funcional',
 'Descripción de Objeto del Gasto',
 'Descripción de Partida Específica',
 'Descripción de Partida Genérica',
 'Descripción de Programa Presupuestario',
 'Descripción de Ramo',
 'Descripción de Subfunción',
 'Descripción de Tipo de Gasto',
 'Descripción de Unidad Responsable',
 'Descripción de la Actividad Institucional',
 'Descripción de la entidad federativa',
 'Descripción de la modalidad del programa presupuestario',
 'Devengado',
 'Ejercicio',
 'Entidad Federativa',
 'Fuente de Financiamiento',
 'Función',
 'Grupo Funcional',
 'Modalidad del Programa presupuestario',
 'Modificado',
 'Objeto del Gasto',
 'Pagado',
 'Partida Específica',
 'Partida Genérica',
 'Programa Presupuestario',
 'Ramo',
 'Subfunción',
 'Tipo 

In [30]:
budget.sample(n=10)

,Ciclo,Ramo,Descripción de Ramo,Unidad Responsable,Descripción de Unidad Responsable,Grupo Funcional,Descripción de Grupo Funcional,Función,Descripción de Función,Subfunción,...,Adefas,Ejercicio,Capitulo,Concepto,Partida Genérica,Partida Específica,Descripción de Capitulo,Descripción de Concepto,Descripción de Partida Genérica,Descripción de Partida Específica
226366,2015,40,Información Nacional Estadística y Geográfica,100,Instituto Nacional de Estadística y Geografía,1,Gobierno,1-8,Otros Servicios Generales,1-8-2,...,0.0,3205983.23,3000,3100,317,31701,Servicios generales,Servicios basicos,"Servicios de acceso de Internet, redes y proce...",Servicios de conducción de señales analógicas ...
55419,2015,6,Hacienda y Crédito Público,AYJ,Comisión Ejecutiva de Atención a Víctimas,1,Gobierno,1-2,Justicia,1-2-4,...,0.0,81824.77,1000,1400,144,14405,Servicios personales,Seguridad social,Aportaciones para seguros,Cuotas para el seguro colectivo de retiro
122802,2015,12,Salud,M00,Comisión Nacional de Arbitraje Médico,2,Desarrollo Social,2-3,Salud,2-3-4,...,0.0,0.00,3000,3100,316,31603,Servicios generales,Servicios basicos,Servicios de telecomunicaciones y satélites,Servicios de Internet
52892,2015,9,Comunicaciones y Transportes,627,Centro SCT Chiapas,3,Desarrollo Económico,3-5,Transporte,3-5-1,...,0.0,19638.80,2000,2400,249,24901,Materiales y suministros,Materiales y articulos de construccion y de re...,Otros materiales y artículos de construcción y...,Otros materiales y artículos de construcción y...
94704,2015,9,Comunicaciones y Transportes,650,Centro SCT Veracruz,3,Desarrollo Económico,3-5,Transporte,3-5-1,...,0.0,0.00,3000,3700,375,37504,Servicios generales,Servicios de traslado y viáticos,Viáticos en el país,Viáticos nacionales para servidores públicos e...
9047,2015,22,Instituto Nacional Electoral,300,Juntas Distritales Ejecutivas,1,Gobierno,1-3,Coordinación de la Política de Gobierno,1-3-6,...,0.0,13461.00,2000,2600,261,26103,Materiales y suministros,"Combustibles, lubricantes y aditivos","Combustibles, lubricantes y aditivos","Combustibles, lubricantes y aditivos para vehí..."
209087,2015,38,Consejo Nacional de Ciencia y Tecnología,90S,"Centro de Investigaciones en Óptica, A.C.",3,Desarrollo Económico,3-8,"Ciencia, Tecnología e Innovación",3-8-1,...,0.0,773310.20,1000,1300,134,13407,Servicios personales,Remuneraciones adicionales y especiales,Compensaciones,Compensaciones adicionales por servicios espec...
223773,2015,40,Información Nacional Estadística y Geográfica,100,Instituto Nacional de Estadística y Geografía,1,Gobierno,1-8,Otros Servicios Generales,1-8-2,...,0.0,10208.00,3000,3500,359,35901,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Servicios de jardinería y fumigación,Servicios de jardinería y fumigación
72037,2015,9,Comunicaciones y Transportes,634,Centro SCT Jalisco,3,Desarrollo Económico,3-5,Transporte,3-5-3,...,442800.0,3618462.44,3000,3400,341,34101,Servicios generales,"Servicios financieros, bancarios y comerciales",Servicios financieros y bancarios,Servicios bancarios y financieros
209781,2015,38,Consejo Nacional de Ciencia y Tecnología,90U,Centro de Investigación en Química Aplicada,3,Desarrollo Económico,3-8,"Ciencia, Tecnología e Innovación",3-8-2,...,0.0,15822970.72,1000,1300,131,13102,Servicios personales,Remuneraciones adicionales y especiales,Primas por años de servicios efectivos prestados,Acreditación por años de servicio en la docenc...


In [31]:
objeto_breakdown = [
    'Ciclo', 
    'Objeto del Gasto', 
    'Capitulo', 'Concepto', 
    'Partida Específica', 
    'Partida Genérica'
]
budget[objeto_breakdown].sample(n=20)

,Ciclo,Objeto del Gasto,Capitulo,Concepto,Partida Específica,Partida Genérica
120364,2015,17102,1000,1700,17102,171
147430,2015,14401,1000,1400,14401,144
151536,2015,35501,3000,3500,35501,355
171434,2015,35101,3000,3500,35101,351
232144,2015,21101,2000,2100,21101,211
91366,2015,37104,3000,3700,37104,371
40485,2015,25401,2000,2500,25401,254
122400,2015,25401,2000,2500,25401,254
237916,2015,35501,3000,3500,35501,355
72862,2015,29601,2000,2900,29601,296


In [32]:
print('Total: missing', len(missing_ids), 'catalog IDs to breakdown the "Objeto del Gasto" column')
print('Tables:', dict(missing_ids.groupby('table').count()['ID']))
print('Years:', dict(missing_ids.groupby('year').count()['ID']))
missing_ids.sample(n=20)

Total: missing 147 catalog IDs to breakdown the "Objeto del Gasto" column
Tables: {'concepto': 7, 'partida_generica': 116, 'partida_especifica': 24}
Years: {2016: 55, 2010: 68, 2012: 22, 2013: 1, 2015: 1}


,ID,table,year
92838,920,partida_generica,2010
2656,250,partida_generica,2010
7712,860,partida_generica,2010
103701,370,partida_generica,2016
103697,330,partida_generica,2016
2662,310,partida_generica,2010
103687,170,partida_generica,2016
8184,720,partida_generica,2010
103661,15906,partida_especifica,2012
103668,34102,partida_especifica,2012


In [33]:
column_mapping

,Column,2010,2011,2012,2013,2014,2015,2016
0,Reasignacion,False,False,False,False,False,False,True
1,Actividad Institucional,True,True,True,True,True,True,True
2,Ejercido,True,True,True,True,False,False,False
3,Tipo de Gasto,True,True,True,True,True,True,True
4,Subfunción,True,True,True,True,True,True,True
5,Descripción de Fuente de Financiamiento,True,True,True,True,True,True,True
6,Modificado,False,False,False,False,True,True,False
7,Aprobado,True,True,True,True,True,True,True
8,Descripción de Unidad Responsable,True,True,True,True,True,True,True
9,Devengado,False,False,False,True,True,True,False


In [34]:
missing_values

,Column,2010,2011,2012,2013,2014,2015,2016
0,Reasignacion,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Actividad Institucional,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ejercido,0.0,1.0,0.0,0.0,NaN,NaN,NaN
3,Tipo de Gasto,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Subfunción,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Descripción de Fuente de Financiamiento,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,Modificado,NaN,NaN,NaN,NaN,0.0,0.0,NaN
7,Aprobado,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8,Descripción de Unidad Responsable,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Devengado,NaN,NaN,NaN,0.0,0.0,0.0,NaN


In [35]:
sums

,Column,2010,2011,2012,2013,2014,2015,2016
0,Ejercido,2.474100e+12,2.695930e+12,2.896331e+12,3.134797e+12,NaN,NaN,NaN
1,Devengado,NaN,NaN,NaN,3.135015e+12,3.426242e+12,3.761997e+12,NaN
2,Aprobado,2.376915e+12,2.538282e+12,2.754868e+12,2.943495e+12,3.334259e+12,3.508463e+12,5.297126e+12
3,Pagado,NaN,NaN,NaN,NaN,3.386609e+12,3.728056e+12,NaN
4,Modificado,NaN,NaN,NaN,NaN,3.427172e+12,3.763467e+12,NaN
5,Adefas,NaN,NaN,NaN,NaN,3.694161e+10,3.112265e+10,NaN
6,Ejercicio,NaN,NaN,NaN,NaN,3.424774e+12,3.760422e+12,NaN


In [36]:
budget.sample(n=20) 

,Ciclo,Ramo,Descripción de Ramo,Unidad Responsable,Descripción de Unidad Responsable,Grupo Funcional,Descripción de Grupo Funcional,Función,Descripción de Función,Subfunción,...,Adefas,Ejercicio,Capitulo,Concepto,Partida Genérica,Partida Específica,Descripción de Capitulo,Descripción de Concepto,Descripción de Partida Genérica,Descripción de Partida Específica
77182,2015,9,Comunicaciones y Transportes,636,Centro SCT Michoacán,3,Desarrollo Económico,3-5,Transporte,3-5-1,...,0.0,0.00,3000,3700,375,37504,Servicios generales,Servicios de traslado y viáticos,Viáticos en el país,Viáticos nacionales para servidores públicos e...
41593,2015,9,Comunicaciones y Transportes,624,Centro SCT Campeche,3,Desarrollo Económico,3-5,Transporte,3-5-1,...,0.0,0.00,2000,2100,216,21601,Materiales y suministros,"Materiales de administracion, emision de docum...",Material de limpieza,Material de limpieza
139169,2015,14,Trabajo y Previsión Social,126,Delegación Federal del Trabajo en Colima,3,Desarrollo Económico,3-1,"Asuntos Económicos, Comerciales y Laborales en...",3-1-2,...,0.0,51142.28,1000,1300,132,13201,Servicios personales,Remuneraciones adicionales y especiales,"Primas de vacaciones, dominical y gratificació...",Primas de vacaciones y dominical
8020,2015,22,Instituto Nacional Electoral,200,Juntas Locales Ejecutivas,1,Gobierno,1-3,Coordinación de la Política de Gobierno,1-3-6,...,0.0,41424.24,1000,1400,143,14302,Servicios personales,Seguridad social,Aportaciones al sistema para el retiro,Depósitos para el ahorro solidario
208172,2015,38,Consejo Nacional de Ciencia y Tecnología,90O,Centro de Investigaciones Biológicas del Noroe...,3,Desarrollo Económico,3-8,"Ciencia, Tecnología e Innovación",3-8-1,...,0.0,836713.00,3000,3300,331,33104,Servicios generales,"Servicios profesionales, cientificos, tecnicos...","Servicios legales, de contabilidad, auditoría ...",Otras asesorías para la operación de programas
15495,2015,4,Gobernación,216,Unidad de Política Interior y Análisis de Info...,1,Gobierno,1-3,Coordinación de la Política de Gobierno,1-3-2,...,0.0,0.00,3000,3100,319,31901,Servicios generales,Servicios basicos,Servicios integrales y otros servicios,Servicios integrales de telecomunicación
145793,2015,16,Medio Ambiente y Recursos Naturales,128,Delegación Federal en Chihuahua,2,Desarrollo Social,2-1,Protección Ambiental,2-1-6,...,0.0,8420.58,2000,2900,291,29101,Materiales y suministros,"Herramientas, refacciones y accesorios menores",Herramientas menores,Herramientas menores
196780,2015,31,Tribunales Agrarios,200,Tribunales Unitarios Agrarios,1,Gobierno,1-2,Justicia,1-2-1,...,0.0,9280.00,3000,3300,336,33603,Servicios generales,"Servicios profesionales, cientificos, tecnicos...","Servicios de apoyo administrativo, traducción,...",Impresiones de documentos oficiales para la pr...
150111,2015,15,"Desarrollo Agrario, Territorial y Urbano",127,Delegación Estatal en Chiapas,2,Desarrollo Social,2-2,Vivienda y Servicios a la Comunidad,2-2-5,...,0.0,20000.00,2000,2900,296,29601,Materiales y suministros,"Herramientas, refacciones y accesorios menores",Refacciones y accesorios menores de equipo de ...,Refacciones y accesorios menores de equipo de ...
227626,2015,40,Información Nacional Estadística y Geográfica,100,Instituto Nacional de Estadística y Geografía,1,Gobierno,1-8,Otros Servicios Generales,1-8-2,...,0.0,1851.01,3000,3700,372,37204,Servicios generales,Servicios de traslado y viáticos,Pasajes terrestres,Pasajes terrestres nacionales para servidores ...


In [43]:
with open(join(ITERATION_FOLDER, BASENAME + '.missing.json')) as file:
    aliases = loads(file.read())
aliases

{'Actividad Institucional': [],
 'Adefas': [],
 'Aprobado': [{'Actividad Institucional': '3',
   'Aprobado': nan,
   'Ciclo': '2011',
   'Descripción de Fuente de Financiamiento': nan,
   'Descripción de Función': 'Ciencia y Tecnología',
   'Descripción de Grupo Funcional': 'Desarrollo Económico',
   'Descripción de Objeto del Gasto': nan,
   'Descripción de Programa Presupuestario': nan,
   'Descripción de Ramo': 'Consejo Nacional de Ciencia y Tecnología',
   'Descripción de Subfunción': 'Investigación Científica',
   'Descripción de Tipo de Gasto': nan,
   'Descripción de Unidad Responsable': 'Centro de Investigación en Materiales Avanzados, S.C.',
   'Descripción de la Actividad Institucional': nan,
   'Descripción de la modalidad del programa presupuestario': nan,
   'Ejercido': nan,
   'Fuente de Financiamiento': nan,
   'Función': '7',
   'Grupo Funcional': '3',
   'Modalidad del Programa presupuestario': nan,
   'Objeto del Gasto': nan,
   'Programa Presupuestario': nan,
   'Ram